<a href="https://colab.research.google.com/github/agalvare/reportportal/blob/master/FineTuneGODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf /content/GODEL
!rm -rf /content/sample_data

In [2]:
requirements = """
absl-py
accelerate
datasets
filelock
jsonlines
nltk
numpy
python-dotenv
rouge_score
six
torch
tqdm
transformers
wandb
fire
flask
flask-Cors
chardet
nltk
beautifulsoup4
zstandard
flashtext
dotmap
nodejs
accelerate
"""
text_file = open("requirements.txt", "w")
n = text_file.write(requirements)
text_file.close()

In [3]:
%%capture
!pip install -r /content/requirements.txt

In [4]:
!rm -rf GODEL
!git clone https://github.com/microsoft/GODEL.git
%cd GODEL/GODEL

Cloning into 'GODEL'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 263 (delta 53), reused 46 (delta 46), pack-reused 195
Receiving objects: 100% (263/263), 51.03 MiB | 23.35 MiB/s, done.
Resolving deltas: 100% (87/87), done.
/content/GODEL/GODEL


In [5]:
import os
import json
try:
  os.mkdir("./data")
except FileExistsError:
  print("File already exist!")
os.environ['TRAIN_SCRIPT_PATH'] = './train.py'
os.environ['MODEL_PATH'] = './GODEL-Base'
os.environ['INPUT_DIR'] = './data/data.json'
os.environ['OUTPUT_DIR'] = './finetuneoutput'
os.environ['EXP_NAME'] = 'learn_tunning'

In [7]:
!pwd

/content/GODEL/GODEL


In [6]:
import json
val =[  {
    "Context": "Please remind me of calling to Jessie at 2PM.",
    "Knowledge": "reminder_contact_name is Jessie, reminder_time is 2PM",
    "Response": "Sure, set the reminder: call to Jesse at 2PM"
  }]
import pathlib
data_path = pathlib.Path("data", "data.json")
if not data_path.parent.exists():
  data_path.parent.mkdir()
with open(data_path, "w") as data_fileobj:
  for record in val:
    # http://ndjson.org/
    data_fileobj.write(json.dumps(val) + "\r\n")


In [8]:
!wget -nc https://bapengstorage.blob.core.windows.net/fileshare/godel_base.tar.gz

--2023-02-14 14:15:06--  https://bapengstorage.blob.core.windows.net/fileshare/godel_base.tar.gz
Resolving bapengstorage.blob.core.windows.net (bapengstorage.blob.core.windows.net)... 20.150.87.36
Connecting to bapengstorage.blob.core.windows.net (bapengstorage.blob.core.windows.net)|20.150.87.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824827060 (787M) [application/gzip]
Saving to: ‘godel_base.tar.gz’

godel_base.tar.gz   100%[===================>] 786.62M  43.9MB/s    in 22s     

2023-02-14 14:15:29 (35.5 MB/s) - ‘godel_base.tar.gz’ saved [824827060/824827060]



In [9]:
!tar -zxvf godel_base.tar.gz

GODEL-Base/
GODEL-Base/tokenizer.json
GODEL-Base/pytorch_model.bin
GODEL-Base/config.json
GODEL-Base/special_tokens_map.json
GODEL-Base/tokenizer_config.json
GODEL-Base/training_args.bin


In [10]:
!python ${TRAIN_SCRIPT_PATH} --model_name_or_path ${MODEL_PATH} \
	--test_file ${INPUT_DIR} \
	--validation_file ${INPUT_DIR} \
	--train_file ${INPUT_DIR} \
	--output_dir ${OUTPUT_DIR} \
	--per_device_train_batch_size=16 \
	--per_device_eval_batch_size=16 \
	--max_target_length 512 \
	--max_length 512 \
	--num_train_epochs 50 \
	--save_steps 10000 \
	--num_beams 5 \
  --preprocessing_num_workers 24 \
  --exp_name ${EXP_NAME}



2023-02-14 14:18:31.013750: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 14:18:31.966131: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-14 14:18:31.966250: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-14 14:18:31.966271: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [ ]:
# !pwd

In [21]:
!python /content/GODEL/GODEL/generate.py --model_name_or_path /content/GODEL/GODEL/GODEL-Base \
                                         --validation_file /content/GODEL/GODEL/data/data.json \
                                         --train_file /content/GODEL/GODEL/data/data.json \
                                         --output_dir /content/GODEL/prediction \
                                         --per_device_eval_batch_size=16 \
                                         --max_target_length 128 \
                                         --max_length 512  \
                                         --preprocessing_num_workers 24 \
                                         --num_beams 5

2023-02-14 14:39:31.424671: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 14:39:32.411762: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-14 14:39:32.411894: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-14 14:39:32.411914: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [1]:
!zip -r /content/file.zip /content/GODEL

  adding: content/GODEL/ (stored 0%)
  adding: content/GODEL/.git/ (stored 0%)
  adding: content/GODEL/.git/packed-refs (deflated 30%)
  adding: content/GODEL/.git/index (deflated 53%)
  adding: content/GODEL/.git/config (deflated 32%)
  adding: content/GODEL/.git/HEAD (stored 0%)
  adding: content/GODEL/.git/info/ (stored 0%)
  adding: content/GODEL/.git/info/exclude (deflated 28%)
  adding: content/GODEL/.git/hooks/ (stored 0%)
  adding: content/GODEL/.git/hooks/pre-merge-commit.sample (deflated 39%)
  adding: content/GODEL/.git/hooks/commit-msg.sample (deflated 44%)
  adding: content/GODEL/.git/hooks/pre-receive.sample (deflated 40%)
  adding: content/GODEL/.git/hooks/update.sample (deflated 68%)
  adding: content/GODEL/.git/hooks/fsmonitor-watchman.sample (deflated 52%)
  adding: content/GODEL/.git/hooks/pre-commit.sample (deflated 45%)
  adding: content/GODEL/.git/hooks/post-update.sample (deflated 27%)
  adding: content/GODEL/.git/hooks/applypatch-msg.sample (deflated 42%)
  addi

In [2]:
from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>